In [87]:
#Importing libraries for API requests.
import shutil
import os
from itertools import repeat
from mpire import WorkerPool
import multiprocessing
import aiofile
import json
import itertools
import sys
import aiohttp
import ssl
from aiohttp import ClientSession, TCPConnector
from aiohttp.client_reqrep import ClientRequest
import asyncio
import nest_asyncio
nest_asyncio.apply()
#loop = asyncio.get_event_loop()
from datetime import datetime, timedelta
import sys,socket
import requests
import json
import pandas as pd
from datetime import datetime
%run -i '/home/juan-david/Documents/data_science/UGA_phd/documents/IMS_data_download/retrieving_data_fonctions_aide.py'

In [2]:
#1.First part.
#Write a bytes object from request.
my_uri = 'https://api.ims.gov.il/v1/envista/stations'
headers={'Authorization': 'ApiToken f058958a-d8bd-47cc-95d7-7ecf98610e47'}
async with aiohttp.ClientSession(headers=headers) as session:
    async with session.get(my_uri) as resp:
        with open('metadata','wb') as f:
            async for chunk in resp.content.iter_chunked(128):
                f.write(chunk)
        #print(sys.getsizeof(r_metadata_json))

In [3]:
#Obtain from bytes json.
file = open("metadata", "r")
r_metadata_json = json.loads(file.read())
type(r_metadata_json)

list

In [4]:
#Print n_lines of request.
print_n_lines_json(r_metadata_json, 25)

[
    {
        "stationId": 2,
        "name": "AVNE ETAN",
        "shortName": "AVNE ETA",
        "stationsTag": "(null)",
        "location": {
            "latitude": 32.817,
            "longitude": 35.763
        },
        "timebase": 10,
        "active": true,
        "owner": "ims",
        "regionId": 8,
        "monitors": [
            {
                "channelId": 1,
                "name": "Rain",
                "alias": null,
                "active": true,
                "typeId": 1,
                "pollutantId": null,
                "units": "mm",
                "description": null
            },
            {


In [5]:
#Obtain consolidated metadata dataframe from request.
ls_1,ls_2,ls_3,ls_4,ls_5  = obtain_from_request_metadata_lists_attributes(r_metadata_json)
df_metadata = obtain_from_metadata_lists_consolidated_df(ls_1,ls_2,ls_3,ls_4,ls_5)
#Filtering only for values for the station 8.
#df_metadata = df_metadata[df_metadata["stationId"].isin([8])]
print(df_metadata.head(15))
print(df_metadata["stationId"].unique())
print(df_metadata["name"].unique())
list_stations = list(df_metadata["stationId"].unique())

   stationId       name latitude longitude channelId  units
0          2  AVNE ETAN   32.817    35.763         1     mm
1          2  AVNE ETAN   32.817    35.763         2  m/sec
2          2  AVNE ETAN   32.817    35.763         3    deg
3          2  AVNE ETAN   32.817    35.763         4  m/sec
4          2  AVNE ETAN   32.817    35.763         5    deg
5          2  AVNE ETAN   32.817    35.763         6    deg
6          2  AVNE ETAN   32.817    35.763         7   degC
7          2  AVNE ETAN   32.817    35.763         8      %
8          2  AVNE ETAN   32.817    35.763         9   degC
9          2  AVNE ETAN   32.817    35.763        10   degC
10         2  AVNE ETAN   32.817    35.763        11   degC
11         2  AVNE ETAN   32.817    35.763        17  m/sec
12         2  AVNE ETAN   32.817    35.763        18  m/sec
13         2  AVNE ETAN   32.817    35.763        19   hhmm
14         6  BET ZAYDA   32.881    35.653         1     mm
[2 6 8 10 11 13 16 17 18 20 21 22 23 24 

In [30]:
#2.Second part.
#Obtain data from multiple stations (requested files are in bits).
year_month_date = '2022/01'
begin_time = datetime.now()
#df_data_new = asyncio.run(obtain_df_data_from_multiple_stations_requests_montly('2021/01',list_stations[0:10],'f058958a-d8bd-47cc-95d7-7ecf98610e47'))
responses = asyncio.run(obtain_df_data_from_multiple_stations_requests_montly(year_month_date,list_stations,'f058958a-d8bd-47cc-95d7-7ecf98610e47'))
#Print spent time.
print(str(datetime.now() - begin_time)+':Total_time')

2
6
8
10
11
13
16
17
18
20
21
22
23
24
25
26
28
29
30
32
33
35
36
41
42
43
44
45
46
54
58
59
60
62
64
65
67
69
73
74
75
77
78
79
82
85
90
98
99
106
107
112
115
121
123
124
178
186
188
202
205
206
207
208
210
211
212
218
224
227
228
232
233
236
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
259
263
265
269
270
271
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
309
310
311
312
313
314
315
316
317
318
319
320
322
323
324
325
327
328
329
330
332
335
336
338
343
344
345
346
349
350
351
353
354
355
366
367
370
373
379
380
381
443
480
498
499
0:15:37.871431:Total_time


In [134]:
year_month_date = '2022/01'
chain = 'IMS_data_'+year_month_date.replace("/", "_")+"/station_"
print(chain)

IMS_data_2022_01/station_


In [155]:
#Obtain data from ten stations.
mylist = [str(i) for i in list_stations][0:10]
keys = ['stationId','datetime','channelId','name','alias','value','status','valid','description','period']
values = [[],[],[],[],[],[],[],[],[],[]]
d = dict(zip(keys,values))
df_data = pd.DataFrame(d)

In [156]:
#Create dataframe of IMS data in regards to specified folder.
begin_time = datetime.now()
for i in mylist:
    print(i)
    try:
        file = open('IMS_data_'+year_month_date.replace("/", "_")+"/station_"+i, "r")
        l = json.loads(file.read())
        ls_1_new, ls_2_new, ls_3_new = obtain_from_request_data_lists_attributes(l)
        df = obtain_from_data_lists_consolidated_df(ls_1_new, ls_2_new, ls_3_new)
        df = df_obtain_period(df,"datetime")
        df_data = pd.concat([df_data,df], ignore_index=True, axis=0)
    except:
        keys = ['stationId','datetime','channelId','name','alias','value','status','valid','description','period']
        values = [[],[],[],[],[],[],[],[],[],[]]
        d = dict(zip(keys,values))
        df_empty = pd.DataFrame(d)
        df_data = pd.concat([df_data,df_empty], ignore_index=True, axis=0)
#Print spent time.
print(str(datetime.now() - begin_time)+':Total_time')

2
6
8
10
11
13
16
17
18
20
0:02:16.143706:Total_time


In [178]:
df_data

,stationId,stationName,latitude,longitude,datetime,datetime_hour,channelId,name,channelUnits,alias,value,status,valid,description,period
6,2,AVNE ETAN,32.817,35.763,2022-01-01T00:10:00+02:00,2022-01-01T00:00:00+02:00,7,TD,degC,None,13.8,1,True,None,2022-01-01
20,2,AVNE ETAN,32.817,35.763,2022-01-01T00:20:00+02:00,2022-01-01T00:00:00+02:00,7,TD,degC,None,13.7,1,True,None,2022-01-01
34,2,AVNE ETAN,32.817,35.763,2022-01-01T00:30:00+02:00,2022-01-01T00:00:00+02:00,7,TD,degC,None,13.7,1,True,None,2022-01-01
48,2,AVNE ETAN,32.817,35.763,2022-01-01T00:40:00+02:00,2022-01-01T00:00:00+02:00,7,TD,degC,None,13.7,1,True,None,2022-01-01
62,2,AVNE ETAN,32.817,35.763,2022-01-01T00:50:00+02:00,2022-01-01T00:00:00+02:00,7,TD,degC,None,13.7,1,True,None,2022-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491681,20,QARNE SHOMERON,32.166,35.099,2022-01-31T23:20:00+02:00,2022-01-31T23:00:00+02:00,8,TD,%,None,8.9,1,True,None,2022-01-31
491694,20,QARNE SHOMERON,32.166,35.099,2022-01-31T23:30:00+02:00,2022-01-31T23:00:00+02:00,8,TD,%,None,9.0,1,True,None,2022-01-31
491707,20,QARNE SHOMERON,32.166,35.099,2022-01-31T23:40:00+02:00,2022-01-31T23:00:00+02:00,8,TD,%,None,8.9,1,True,None,2022-01-31
491720,20,QARNE SHOMERON,32.166,35.099,2022-01-31T23:50:00+02:00,2022-01-31T23:00:00+02:00,8,TD,%,None,8.9,1,True,None,2022-01-31


In [179]:
#Here we could explore and check the resolution of the data. (resolution of 10 mins)
#print dataframe properties.
print(len(df_data["period"].unique()))
print(len(df_data["stationId"].unique()))
print(df_data["name"].unique())
print(df_data.shape)
df_data.head(5)

32
8
['TD']
(35079, 15)


,stationId,stationName,latitude,longitude,datetime,datetime_hour,channelId,name,channelUnits,alias,value,status,valid,description,period
6,2,AVNE ETAN,32.817,35.763,2022-01-01T00:10:00+02:00,2022-01-01T00:00:00+02:00,7,TD,degC,None,13.8,1,True,None,2022-01-01
20,2,AVNE ETAN,32.817,35.763,2022-01-01T00:20:00+02:00,2022-01-01T00:00:00+02:00,7,TD,degC,None,13.7,1,True,None,2022-01-01
34,2,AVNE ETAN,32.817,35.763,2022-01-01T00:30:00+02:00,2022-01-01T00:00:00+02:00,7,TD,degC,None,13.7,1,True,None,2022-01-01
48,2,AVNE ETAN,32.817,35.763,2022-01-01T00:40:00+02:00,2022-01-01T00:00:00+02:00,7,TD,degC,None,13.7,1,True,None,2022-01-01
62,2,AVNE ETAN,32.817,35.763,2022-01-01T00:50:00+02:00,2022-01-01T00:00:00+02:00,7,TD,degC,None,13.7,1,True,None,2022-01-01


In [147]:
#df_data_original = df_data

In [159]:
#df_data = df_data_original

In [160]:
#Check for one station, one day data, and one variable.
#df_data = df_data.copy()
#df_data = df_data[df_data["stationId"]==2]
#df_data = df_data[df_data["period"].isin(["2022-01-01","2022-01-02"])]
#df_data = df_data[df_data["name"]=="TD"]
#df_data = df_data[df_data["datetime"].isin(["2022-01-01T02:20:00+02:00","2022-01-01T02:30:00+02:00","2022-01-02T01:20:00+02:00","2022-01-02T01:30:00+02:00"])]
#print(len(df_data["datetime"].unique()))
#df_data.head(24)

In [180]:
#Third part.
#3.Mapping values from df_metadata to df_data to obtain channelId units, and stationId names and location components.
#Create mappings.
mp_stationId_name = create_column_mapping(df_metadata,"stationId","name")
mp_stationId_latitude = create_column_mapping(df_metadata,"stationId","latitude")
mp_stationId_longitude = create_column_mapping(df_metadata,"stationId","longitude")
mp_channelId_units = create_column_mapping(df_metadata,"channelId","units")

#Create new columns.
df_data = df_new_column_from_mapping(df_data, mp_stationId_name, "stationName", "stationId")
df_data = df_new_column_from_mapping(df_data, mp_stationId_latitude, "latitude", "stationId")
df_data = df_new_column_from_mapping(df_data, mp_stationId_longitude, "longitude", "stationId")
df_data = df_new_column_from_mapping(df_data, mp_channelId_units, "channelUnits", "channelId")
df_data['datetime_hour']= df_data['datetime'].apply(lambda x: calculate_hour_from_date(x))

#Organize the displayment of columns in df.
df_data = df_data[["stationId","stationName","latitude","longitude","datetime","datetime_hour","channelId","name","channelUnits","alias","value","status","valid","description","period"]]

print(df_data.head(10))
print(df_data.shape)
print(df_data["stationId"].unique())
print(df_data["period"].unique())

    stationId stationName  latitude  longitude                   datetime  \
6           2   AVNE ETAN    32.817     35.763  2022-01-01T00:10:00+02:00   
20          2   AVNE ETAN    32.817     35.763  2022-01-01T00:20:00+02:00   
34          2   AVNE ETAN    32.817     35.763  2022-01-01T00:30:00+02:00   
48          2   AVNE ETAN    32.817     35.763  2022-01-01T00:40:00+02:00   
62          2   AVNE ETAN    32.817     35.763  2022-01-01T00:50:00+02:00   
76          2   AVNE ETAN    32.817     35.763  2022-01-01T01:00:00+02:00   
90          2   AVNE ETAN    32.817     35.763  2022-01-01T01:10:00+02:00   
104         2   AVNE ETAN    32.817     35.763  2022-01-01T01:20:00+02:00   
118         2   AVNE ETAN    32.817     35.763  2022-01-01T01:30:00+02:00   
132         2   AVNE ETAN    32.817     35.763  2022-01-01T01:40:00+02:00   

                 datetime_hour channelId name channelUnits alias value status  \
6    2022-01-01T00:00:00+02:00         7   TD         degC  None  13.8 

In [151]:
#df_data_original_2 = df_data

In [162]:
#df_data = df_data_original_2
#df_data.head(5)

In [104]:
#Structurized dataframe data.
#df_stations = df_data.pivot_table(values='value', index=['period','stationId','latitude','longitude'], columns='name', aggfunc='first')
#df_stations.columns = [''.join(tup).strip() for tup in df_stations.columns.values]
#df_stations = df_stations.reset_index()
#print(df_stations.shape)
#print(len(df_stations['stationId'].unique()))
#print(len(df_stations['period'].unique()))
#df_stations.head(5)

In [181]:
#Aggregation by mean.
df_stations = df_data.pivot_table(values='value', index=['datetime_hour','period','stationId','latitude','longitude'], columns='name', aggfunc='mean')
df_stations.columns = [''.join(tup).strip() for tup in df_stations.columns.values]
df_stations = df_stations.reset_index()
print(df_stations.shape)
print(len(df_stations['stationId'].unique()))
print(len(df_stations['datetime_hour'].unique()))
df_stations.head(5)

(5855, 6)
8
745


,datetime_hour,period,stationId,latitude,longitude,TD
0,2022-01-01T00:00:00+02:00,2022-01-01,2,32.817,35.763,13.72
1,2022-01-01T00:00:00+02:00,2022-01-01,6,32.881,35.653,16.52
2,2022-01-01T00:00:00+02:00,2022-01-01,8,32.704,35.584,18.34
3,2022-01-01T00:00:00+02:00,2022-01-01,10,33.133,35.783,8.96
4,2022-01-01T00:00:00+02:00,2022-01-01,11,32.683,35.516,16.70


In [182]:
df_stations_check = df_stations.copy()
df_stations_check = df_stations_check[df_stations_check["stationId"]==2]
df_stations_check = df_stations_check[df_stations_check["period"]=="2022-01-01"]
print(len(df_stations_check["datetime_hour"].unique()))
df_stations_check.head(30)

24


,datetime_hour,period,stationId,latitude,longitude,TD
0,2022-01-01T00:00:00+02:00,2022-01-01,2,32.817,35.763,13.720000
7,2022-01-01T01:00:00+02:00,2022-01-01,2,32.817,35.763,13.483333
14,2022-01-01T02:00:00+02:00,2022-01-01,2,32.817,35.763,11.750000
22,2022-01-01T03:00:00+02:00,2022-01-01,2,32.817,35.763,11.066667
29,2022-01-01T04:00:00+02:00,2022-01-01,2,32.817,35.763,10.616667
36,2022-01-01T05:00:00+02:00,2022-01-01,2,32.817,35.763,10.233333
43,2022-01-01T06:00:00+02:00,2022-01-01,2,32.817,35.763,10.066667
50,2022-01-01T07:00:00+02:00,2022-01-01,2,32.817,35.763,9.883333
57,2022-01-01T08:00:00+02:00,2022-01-01,2,32.817,35.763,10.150000
64,2022-01-01T09:00:00+02:00,2022-01-01,2,32.817,35.763,11.550000


In [183]:
#Retrieve general information of df_data dataframe.
df_data_info = give_df_info(df_stations)
df_data_info

,Variable_names,Data_types,Nan_percent,Unique_values
0,datetime_hour,object,0.0,745
1,period,object,0.0,32
2,stationId,int64,0.0,8
3,latitude,float64,0.0,real_variable
4,longitude,float64,0.0,real_variable
5,TD,float64,0.0,real_variable


In [190]:
#Create folder of storage.
parent_dir = os.getcwd()
path = os.path.join(parent_dir, 'df_IMS_data_'+year_month_date.replace("/", "_")) 
if os.path.exists(path):
        shutil.rmtree(path)
os.mkdir(path) 
#Export dataframe with results.
df_stations.to_csv('df_IMS_data_'+year_month_date.replace("/", "_")+"/df_stations.csv", index = False)

In [185]:
#Import dataframe of results.
df_stations = pd.read_csv ('df_IMS_data_'+year_month_date.replace("/", "_")+"/df_stations.csv")

In [186]:
#Print properties.
print(df_stations.shape)
print(len(df_stations['stationId'].unique()))
print(df_stations['stationId'].unique())
print(len(df_stations['period'].unique()))
df_stations.head(5)

(5855, 6)
8
[ 2  6  8 10 11 13 16 20]
32


,datetime_hour,period,stationId,latitude,longitude,TD
0,2022-01-01T00:00:00+02:00,2022-01-01,2,32.817,35.763,13.72
1,2022-01-01T00:00:00+02:00,2022-01-01,6,32.881,35.653,16.52
2,2022-01-01T00:00:00+02:00,2022-01-01,8,32.704,35.584,18.34
3,2022-01-01T00:00:00+02:00,2022-01-01,10,33.133,35.783,8.96
4,2022-01-01T00:00:00+02:00,2022-01-01,11,32.683,35.516,16.70


In [189]:
df_stations.shape

(5855, 6)